In [ ]:
# no need to restart kernel after code changes (useful to separate code to modules)
%load_ext autoreload
%autoreload 2

In [ ]:
# import from huggingface roneneldan/TinyStories-1M
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from nnsight import LanguageModel
import torch as t
# load garbage collection and empty cache 
import gc

# UNEMBEDDING BASELINE

In [ ]:
def clean():
    gc.collect()
    # empty_cache()

model_id = "roneneldan/TinyStories-1M"
# model_id = "EleutherAI/gpt-j-6b"

device = 'cuda' if torch.cuda.is_available() else 'cpu'

load_in_8bit = False

try:
    del llm
    clean()
    llm = LanguageModel(model_id, device_map=device, load_in_8bit=load_in_8bit)
    tokenizer = llm.tokenizer
except:
    llm = LanguageModel(model_id, device_map=device, load_in_8bit=load_in_8bit)
    tokenizer = llm.tokenizer

prompt_trial = "1:10,2:20,3:"
prompt_trial = "my name is "

In [ ]:

def embeddings_to_texts_baseline(embeddings, model, tokenizer, skip_special_tokens=True):
    """
    Map input embeddings (batch, seq_len, emb_dim) → list of decoded strings.
    """
    # 1) Project embeddings to vocab logits
    logits = model.lm_head(embeddings.to(device))           # (batch, seq_len, vocab_size)
    # 2) Greedy decode: pick highest logit per position
    token_ids = torch.argmax(logits, dim=-1)     # (batch, seq_len)
    # 3) Transform each sequence of IDs into text
    texts = tokenizer.batch_decode(token_ids, skip_special_tokens=skip_special_tokens)
    return logits, texts

def get_next_token_prediction(embeddings, model, tokenizer, skip_special_tokens=True):
    # 1) Project embeddings to vocab logits
    logits = model.lm_head(embeddings)           # (batch, seq_len, vocab_size)
    # 2) Greedy decode: pick highest logit per position
    token_ids = torch.argmax(logits, dim=-1)[:,-1]     # (batch, seq_len)
    # 3) Transform each sequence of IDs into text
    texts = tokenizer.batch_decode(token_ids, skip_special_tokens=skip_special_tokens)
    return texts

@t.inference_mode()
def get_residual_output(prompt, layer_idx, llm, normalize = False):

    assert hasattr(llm, 'transformer'), "The model does not have a transformer attribute."
    assert hasattr(llm.transformer, 'h'), "The transformer does not have a 'h' attribute for layers."


    with llm.trace(prompt):
        residual_output = llm.transformer.h[layer_idx].output[0].save()  # Save the output of the layer for inspection

    if normalize: # FIXME: check if this is the correct way to normalize
        residual_output = llm.transformer.ln_f(residual_output)
    
    if llm.device.type == "cuda":
        residual_output = residual_output.detach().cpu()
        clean()
    
    return residual_output

@t.inference_mode()
def get_embeddings(prompt,llm):

    assert hasattr(llm, 'transformer'), "The model does not have a transformer attribute."
    assert hasattr(llm.transformer, 'drop'), "The transformer does not have a 'drop' attribute for input embeddings."
    
    with llm.trace(prompt):
        input_embeddings = llm.transformer.drop.input.save()

    if llm.device.type == "cuda":
        input_embeddings = input_embeddings.detach().to("cpu")
        clean()
    return input_embeddings
        


In [ ]:
@t.inference_mode()
def get_one_token_h(prompt, layer_idx, model, tokenizer):
    llm = LanguageModel(model, tokenizer=tokenizer) if not isinstance(model, LanguageModel) else model

    # check that the prompt is a single token
    assert len(tokenizer(prompt)["input_ids"]) == 1, "The prompt must be a single token."
    # get the hidden representation of the prompt at layer_idx
    with llm.trace(prompt):
        hidden = llm.transformer.h[layer_idx].output.save()
    if llm.device.type == "cuda":
        hidden = hidden.detach().cpu()
        clean()
    return hidden

@t.inference_mode()
def get_hidden_representation(prompt, layer_idx, model, tokenizer):
    llm = LanguageModel(model, tokenizer=tokenizer) if not isinstance(model, LanguageModel) else model
    with llm.trace(prompt):
        hidden = llm.transformer.h[layer_idx].output.save()
    if llm.device.type == "cuda":
        hidden = hidden.detach().cpu()
        clean()
    return hidden[0].squeeze()
    

### TRYING THE DECODING

In [ ]:
# make a for loop where for each layer you try the embeddings_to_texts_baseline for each hidden state output
full_text = False
print(f"{prompt_trial=} \n\n")
if full_text:   
    reversed_embeddings = get_embeddings(prompt_trial, llm)
    logits, texts = embeddings_to_texts_baseline(reversed_embeddings, llm, tokenizer)
    print(f"Reversed Embeddings for the prompt: {texts= } \n\n")
else:
    next_token_prediction = get_next_token_prediction(get_embeddings(prompt_trial, llm), llm, tokenizer)
    print(f"Next token prediction for the prompt: {next_token_prediction= } \n\n")


print(f"Iterating through each layer's output for the prompt: {prompt_trial}\n")
for layer_idx in range(len(llm.transformer.h)):
    residual_output = get_residual_output(prompt_trial, layer_idx, llm, True)
    if full_text:
        logits, texts = embeddings_to_texts_baseline(residual_output, llm, tokenizer)
        print(f"Layer {layer_idx} output texts: {texts}")
    else:
        next_token_prediction = get_next_token_prediction(residual_output, llm, tokenizer)
        print(f"Layer {layer_idx} next token prediction: {next_token_prediction}")

In [ ]:
import torch

def get_whole1(
    prompt: str,
    llm: torch.nn.Module,
    tokenizer,
    layer_idx: int
) -> torch.Tensor:
    # 1) Tokenize (and move to same device as the model).
    device = next(llm.parameters()).device
    encoded = tokenizer(prompt, return_tensors="pt")
    input_ids = encoded["input_ids"].to(device)      # shape (1, seq_len)
    attention_mask = encoded.get("attention_mask", None)
    if attention_mask is not None:
        attention_mask = attention_mask.to(device)

    # 2) Forward pass under no_grad, requesting hidden_states
    with torch.no_grad():
        outputs = llm(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
        )
        hidden_states = outputs.hidden_states
        # hidden_states is a tuple: (layer0, layer1, ..., layerN),
        # where layer0 = embeddings output, layer1 = first transformer block, etc.

        # 3) Index into [0, -1, :] to get the last token for this single‐batch example
        #    hidden_states[layer_idx] has shape (batch_size=1, seq_len, hidden_size).
        return hidden_states[layer_idx][0]


In [ ]:
# make a for loop where for each layer you try the embeddings_to_texts_baseline for each hidden state output
prompt = 'my name is george'
prefix = 'my name'

for layer_idx in range(len(llm.transformer.h)):
    # residual_output1 = get_residual_output(prompt, layer_idx, llm, True)
    # residual_output2 = get_residual_output(prefix, layer_idx, llm, True)

    # residual_output1 = get_hidden_representation(prompt, layer_idx, llm, tokenizer)
    # residual_output2 = get_hidden_representation(prefix, layer_idx, llm, tokenizer)

    residual_output1 = get_whole1(prompt, llm, tokenizer, layer_idx)
    residual_output2 = get_whole1(prefix, llm, tokenizer, layer_idx)

    print(residual_output1.shape)
    print(residual_output2.shape)
    print(torch.mean((residual_output1[:2,:] - residual_output2) ** 2))
    print()

    

# GRADIENT BASED ALGORITHM

In [ ]:
from utils import compute_last_token_embedding_grad

In [ ]:
# function that takes as input a tokenizer and an embedding matrix and returns a random token and its embedding
def get_random_token_and_embedding(tokenizer, embedding_matrix):
    # Get the vocabulary size
    vocab_size = embedding_matrix.shape[0]
    
    # Generate a random token ID
    random_token_id = torch.randint(0, vocab_size, (1,)).item()
    
    # Get the corresponding embedding
    embedding = embedding_matrix[random_token_id]
    
    # Decode the token ID to get the token string
    token_string = tokenizer.decode(random_token_id)
    
    return token_string, embedding

# function that takes as input a tokenizer and an embedding matrix and returns a random token and its embedding
def get_random_token_id_and_embedding(embedding_matrix):
    # Get the vocabulary size
    vocab_size = embedding_matrix.shape[0]
    
    # Generate a random token ID
    random_token_id = torch.randint(0, vocab_size, (1,)).item()
    
    # Get the corresponding embedding
    embedding = embedding_matrix[random_token_id]

    
    return random_token_id, embedding

In [ ]:
from tqdm import tqdm
# take as input llm, tokenizer, embedding matrix, an hidden representation, layer_idx, n_iterations, step_size, 


tokenizer = AutoTokenizer.from_pretrained("roneneldan/TinyStories-1M")
model = AutoModelForCausalLM.from_pretrained("roneneldan/TinyStories-1M", output_hidden_states=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
prompt = "George"
h = get_one_token_h(prompt, 4, model,tokenizer)
embedding_matrix = model.get_input_embeddings().weight  # (vocab_size, hidden_size)
gamma = 1  # step size for gradient descent
layer_idx = 4  # layer at which we want to compute the gradient
n_iterations = 100  # number of iterations for gradient descent

In [ ]:
prompt = "George"
h = get_one_token_h(prompt, 4, model,tokenizer)
embedding_matrix = model.get_input_embeddings().weight  # (vocab_size, hidden_size)
gamma = 1  # step size for gradient descent
layer_idx = 4  # layer at which we want to compute the gradient
n_iterations = 100  # number of iterations for gradient descent

# make a guess for (e_0,y_0)
y_i, x_i_plus_1= get_random_token_and_embedding(tokenizer, embedding_matrix)

# loop through the n_iterations
for iteration in (bar:=tqdm(range(n_iterations))):
    bar.set_postfix_str(f"Current guess: {y_i}")
    # compute the embedding of the current guess
    # P_hat_i = embedding_matrix[tokenizer.encode(y_i)[0]]  # (hidden_size,)
    
    # compute the gradient of the oracle with respect to e_i -> inner for
    grad_oracle = compute_last_token_embedding_grad(
        y=torch.tensor(tokenizer.encode(y_i), dtype=torch.long), # turn into x_i_plus_1
        llm=model,
        layer_idx=layer_idx,
        h=h
    )  # (hidden_size,)
    bar.set_postfix_str(f"Current guess: {y_i}, Gradient norm: {grad_oracle.norm().item():.4f}")

    # update the guess using gradient descent
    x_i_plus_1 = x_i_plus_1 - gamma * grad_oracle  # (hidden_size,)
    
    # find the closest token in the embedding space to e_i_plus_1
    distances = torch.norm(embedding_matrix - x_i_plus_1, dim=1)  # (vocab_size,)
    closest_token_id = torch.argmin(distances).item()
    y_i = tokenizer.decode(closest_token_id)

    # make a summary via prints
    print(f"Iteration {iteration + 1}/{n_iterations}:")
    print(f"  Current guess: {y_i}")
    print(f"  Gradient norm: {grad_oracle.norm().item():.4f}")





In [ ]:
prompt = "George"
n = 100
# tokenizer it but return text
h = get_hidden_representation(prompt, layer_idx, model, tokenizer)
size = 1 if len(h.shape)==1 else h.shape[0]

# discovered_tokens = []

In [ ]:
# tokenizer.tokenize("my name is George")
# tokenizer.encode("name")
# discovered_tokens = ["my", " name", " is"]
# # merge discovered tokens and y_i in a list and a string
# lista = discovered_tokens + [y_i]
# stringa = "".join(lista)
# y_i, discovered_tokens, lista, stringa, tokenizer.tokenize(stringa)
# y_extend = "".join((discovered_tokens + [y_i]))
# print(f"Extended guess: {y_extend}")
# encoded_y = torch.tensor(tokenizer.encode(y_extend), dtype=torch.long) 
# grad_oracle = compute_last_token_embedding_grad(
#         y=encoded_y, # turn into x_i_plus_1
#         llm=model,
#         layer_idx=layer_idx,
#         h=h
#     )

In [ ]:
import torch
import torch.nn.functional as F

import torch
from typing import Optional

def get_whole(
    prompt: str,
    llm: torch.nn.Module,
    tokenizer,
    layer_idx: int,
    input_ids: Optional[torch.Tensor] = None,
    grad: bool = False
) -> torch.Tensor:
    """
    Tokenize `prompt`, do a forward pass with output_hidden_states=True,
    and return the hidden vector of the *last* token at layer `layer_idx`.

    Args:
        prompt (str): the input string, e.g. "Harry".
        llm (nn.Module): a HuggingFace‐style model (with embeddings + hidden_states).
        tokenizer: the corresponding tokenizer for `llm`.
        layer_idx (int): which hidden‐layer index to extract (0=embeddings, 1=first block, etc.)

    Returns:
        Tensor of shape (hidden_size,) = the last‐token hidden state at `layer_idx`,
        computed under torch.no_grad().
    """
    # 1) Tokenize (and move to same device as the model).
    device = next(llm.parameters()).device
    if input_ids is None:
        encoded = tokenizer(prompt, return_tensors="pt")
        input_ids = encoded["input_ids"].to(device)      # shape (1, seq_len)
    
    if not grad:
        # Forward under no_grad and detach before returning
        with torch.no_grad():
            outputs = llm(
                input_ids=input_ids,
                output_hidden_states=True
            )
            hidden_states = outputs.hidden_states
            h = hidden_states[layer_idx][0]  # shape = (seq_len, hidden_size)

        return h.detach()
    else:
        # Forward normally, so gradients can flow
        outputs = llm(
            input_ids=input_ids,
            output_hidden_states=True
        )
        hidden_states = outputs.hidden_states
        h = hidden_states[layer_idx][0]  # shape = (seq_len, hidden_size)
        return h


def compute_last_token_embedding_grad(
    y: torch.LongTensor,
    llm: torch.nn.Module,
    layer_idx: int,
    h_target: torch.Tensor
):
    device = next(llm.parameters()).device
    y = y.to(device)
    h_target = h_target.to(device)

    emb_layer = llm.get_input_embeddings()
    if not emb_layer.weight.requires_grad:
        emb_layer.weight.requires_grad_(True)

    llm.zero_grad()
    emb_layer.zero_grad()

    with torch.set_grad_enabled(True):
        h_last = get_whole('', llm, tokenizer, layer_idx, y.unsqueeze(0), grad=True)[-1]
        diff = h_last - h_target
        loss = torch.dot(diff, diff)
        loss.backward()

    last_token_id = y[-1].item()
    grad_last_embedding = emb_layer.weight.grad[last_token_id].detach().clone() # TODO: Is this gradient with respect to the input or not?

    llm.zero_grad()
    emb_layer.zero_grad()

    return grad_last_embedding, loss.item()


## Vanilla Algo

In [ ]:
embedding_matrix = model.get_input_embeddings().weight  # (vocab_size, hidden_size)

prompt = "my name is george and i am living here in greece, i am 20 years old and my secret is that i am in love"
layer_idx = 8  # layer at which we want to compute the gradient

# h = get_hidden_representation(prompt, layer_idx, model, tokenizer)
h = get_whole(prompt, model, tokenizer, layer_idx)
if h.dim() == 1:
    h = h.unsqueeze(0)

gamma = 1e-0  # step size for gradient descent
n_iterations = 5000  # number of iterations for gradient descent

discovered_ids = []
for j in range(h.size(0)):
    # an idea here is to initialize with feeding an LLM with the prompt so far
    # and getting the next token, more expensive, likely to work much better
    y_i_id, x_i_plus_1 = get_random_token_id_and_embedding(embedding_matrix)

    for iteration in (bar := tqdm(range(n_iterations), desc=f'Token [{j:2d}/{h.size(0):2d}]')):
        input_ids = torch.tensor(discovered_ids + [y_i_id], dtype=torch.long)

        h_pred = get_whole('', model, tokenizer, layer_idx, input_ids.unsqueeze(0), grad=False)[-1]

        if torch.sum((h_pred - h[:h_pred.size(0),:]) ** 2) <= 1e-10:
            print('Early stopping')
            break

        grad_oracle, loss = compute_last_token_embedding_grad(
            y=input_ids, # turn into x_i_plus_1
            llm=model,
            layer_idx=layer_idx,
            h_target=h[j]
        ) # TODO: Dont use tokens, rather use previous embeddings + x_i_plus_1
        # TODO: non-zero loss even when having the correct y_i_id
        
        string_so_far = tokenizer.decode(input_ids.cpu().tolist(), skip_special_tokens=True)

        bar.set_postfix_str(f"Loss: {loss:.2e} - Gradient norm: {grad_oracle.norm().item():.2e} - String: {string_so_far}")
        # print('', flush=True)
        
        if string_so_far == prompt:
            break

        if loss < 1e-6:
            # print(f"Found token {y_i} with small L2 norm: {torch.norm(h[j] - embedding_matrix[closest_token_id]).item():.4f}")
            break

        # update the guess using gradient descent
        x_i_plus_1 = x_i_plus_1 - gamma * grad_oracle  # (hidden_size,)
        
        # find the closest token in the embedding space to e_i_plus_1
        distances = torch.norm(embedding_matrix - x_i_plus_1, dim=1)  # (vocab_size,)
        y_i_id = int(torch.argmin(distances))


    
    discovered_ids.append(y_i_id)

final_string = tokenizer.decode(input_ids.cpu().tolist(), skip_special_tokens=True)
print(f"Final discovered tokens: {final_string}")


In [ ]:
import torch
from tqdm import tqdm

embedding_matrix = model.get_input_embeddings().weight  # shape (vocab_size, hidden_size)

prompt = "Helenaki is hereeee and is a cutie"
layer_idx = 4  # layer at which we want to compute the gradient

# 1) Get the “target” hidden states for each token in `prompt`
h = get_whole(prompt, model, tokenizer, layer_idx, grad=False)
if h.dim() == 1:
    h = h.unsqueeze(0)  # now shape (seq_len, hidden_size)

# 2) Adam hyperparameters
gamma = 1e-1        # base learning rate (you can tune this)
beta1 = 0.9
beta2 = 0.999
eps = 1e-8
n_iterations = 5000

discovered_ids = []

for j in range(h.size(0)):
    # ——— Initialize a random token and its “fake” embedding guess ———
    y_i_id, x_i_plus_1 = get_random_token_id_and_embedding(embedding_matrix)
    # x_i_plus_1 has shape (hidden_size,), on same device as embedding_matrix

    # 3) Initialize Adam buffers m, v (shape = hidden_size) to zero
    m = torch.zeros_like(x_i_plus_1)
    v = torch.zeros_like(x_i_plus_1)

    for iteration in (bar := tqdm(range(n_iterations), desc=f"Token [{j:2d}/{h.size(0):2d}]")):
        # Build the current hypothesis of token‐IDs:
        input_ids = torch.tensor(discovered_ids + [y_i_id], dtype=torch.long, device=embedding_matrix.device)

        # 4) Compute gradient ∇_x L₂( h_last, h_target ) at layer_idx
        grad_oracle, loss = compute_last_token_embedding_grad(
            y=input_ids,          # tensor of shape (seq_len,)
            llm=model,
            layer_idx=layer_idx,
            h_target=h[j]
        )

        # 5) Logging / progress
        string_so_far = tokenizer.decode(input_ids.cpu().tolist(), skip_special_tokens=True)
        bar.set_postfix_str(
            f"Loss: {loss:.4f} | ‖grad‖: {grad_oracle.norm().item():.4f} | '{string_so_far}'"
        )

        if string_so_far == prompt:
            break
        if grad_oracle.norm(p=2) < 1e-4:
            # gradient is essentially zero → we’re “close enough”
            break

        # ——— Adam update on x_i_plus_1 ———
        t = iteration + 1
        # 6a) Update biased first moment estimate
        m = beta1 * m + (1 - beta1) * grad_oracle
        # 6b) Update biased second moment estimate (elementwise square)
        v = beta2 * v + (1 - beta2) * (grad_oracle * grad_oracle)
        # 6c) Compute bias‐corrected m_hat, v_hat
        m_hat = m / (1 - beta1**t)
        v_hat = v / (1 - beta2**t)
        # 6d) Take Adam step
        x_i_plus_1 = x_i_plus_1 - gamma * (m_hat / (v_hat.sqrt() + eps))

        # 7) Project x_i_plus_1 back to the nearest token in embedding space
        #    (Choose the token whose embedding row is closest in L₂ distance)
        distances = torch.norm(embedding_matrix - x_i_plus_1.unsqueeze(0), dim=1)  # (vocab_size,)
        y_i_id = int(torch.argmin(distances))

    # End of iteration loop for this token j
    discovered_ids.append(y_i_id)

# After all tokens are “discovered”:
final_string = tokenizer.decode(torch.tensor(discovered_ids, dtype=torch.long), skip_special_tokens=True)
print(f"Final discovered tokens: {final_string}")

In [ ]:

h = get_whole(prompt, model, tokenizer, layer_idx, grad=False)

h.shape

In [ ]:
prompt = "George"
h = get_one_token_h(prompt, 4, model,tokenizer)[0]
embedding_matrix = model.get_input_embeddings().weight  # (vocab_size, hidden_size)
gamma = 1  # step size for gradient descent
layer_idx = 4  # layer at which we want to compute the gradient
n_iterations = 100  # number of iterations for gradient descent

# make a guess for (e_0,y_0)
y_i, x_i_plus_1= get_random_token_and_embedding(tokenizer, embedding_matrix)

# loop through the n_iterations
for iteration in (bar:=tqdm(range(n_iterations))):
    bar.set_postfix_str(f"Current guess: {y_i}")
    # compute the embedding of the current guess
    # P_hat_i = embedding_matrix[tokenizer.encode(y_i)[0]]  # (hidden_size,)
    
    # compute the gradient of the oracle with respect to e_i -> inner for
    grad_oracle, _ = compute_last_token_embedding_grad(
        y=torch.tensor(tokenizer.encode(y_i), dtype=torch.long), # turn into x_i_plus_1
        llm=model,
        layer_idx=layer_idx,
        h=h
    )  # (hidden_size,)
    # bar.set_postfix_str(f"Current guess: {y_i}, Gradient norm: {grad_oracle.norm().item():.4f}")

    # update the guess using gradient descent
    x_i_plus_1 = x_i_plus_1 - gamma * grad_oracle  # (hidden_size,)
    
    # find the closest token in the embedding space to e_i_plus_1
    distances = torch.norm(embedding_matrix - x_i_plus_1, dim=1)  # (vocab_size,)
    closest_token_id = torch.argmin(distances).item()
    y_i = tokenizer.decode(closest_token_id)

    # make a summary via prints
    print(f"Current guess: {y_i} - Target: {prompt} - Gradient norm: {grad_oracle.norm().item():.4f}")





## Boosted Algo

In [ ]:
embedding_matrix = model.get_input_embeddings().weight  # (vocab_size, hidden_size)

prompt = "my name is george and i am living here in greece, i am 20 years old and my secret is that i am in love"
layer_idx = 8  # layer at which we want to compute the gradient

# h = get_hidden_representation(prompt, layer_idx, model, tokenizer)
h = get_whole(prompt, model, tokenizer, layer_idx)
if h.dim() == 1:
    h = h.unsqueeze(0)

gamma = 1e-0  # step size for gradient descent
n_iterations = 5000  # number of iterations for gradient descent

discovered_ids = []
for j in range(h.size(0)):
    # an idea here is to initialize with feeding an LLM with the prompt so far
    # and getting the next token, more expensive, likely to work much better
    y_i_id, x_i_plus_1 = get_random_token_id_and_embedding(embedding_matrix)

    for iteration in (bar := tqdm(range(n_iterations), desc=f'Token [{j:2d}/{h.size(0):2d}]')):
        input_ids = torch.tensor(discovered_ids + [y_i_id], dtype=torch.long)

        h_pred = get_whole('', model, tokenizer, layer_idx, input_ids.unsqueeze(0), grad=False)[-1]

        if torch.sum((h_pred - h[:h_pred.size(0),:]) ** 2) <= 1e-10:
            print('Early stopping')
            break

        grad_oracle, loss = compute_last_token_embedding_grad(
            y=input_ids, # turn into x_i_plus_1
            llm=model,
            layer_idx=layer_idx,
            h_target=h[j]
        ) # TODO: Dont use tokens, rather use previous embeddings + x_i_plus_1
        # TODO: non-zero loss even when having the correct y_i_id
        
        string_so_far = tokenizer.decode(input_ids.cpu().tolist(), skip_special_tokens=True)

        bar.set_postfix_str(f"Loss: {loss:.2e} - Gradient norm: {grad_oracle.norm().item():.2e} - String: {string_so_far}")
        
        if string_so_far == prompt:
            break

        if loss < 1e-8:
            # print(f"Found token {y_i} with small L2 norm: {torch.norm(h[j] - embedding_matrix[closest_token_id]).item():.4f}")
            break

        # update the guess using gradient descent
        x_i_plus_1 = x_i_plus_1 - gamma * grad_oracle  # (hidden_size,)
        
        # find the closest token in the embedding space to e_i_plus_1
        distances = torch.norm(embedding_matrix - x_i_plus_1, dim=1)  # (vocab_size,)
        y_i_id = int(torch.argmin(distances))


    
    discovered_ids.append(y_i_id)

final_string = tokenizer.decode(input_ids.cpu().tolist(), skip_special_tokens=True)
print(f"Final discovered tokens: {final_string}")


## Debugging

In [ ]:
embedding_matrix = model.get_input_embeddings().weight  # (vocab_size, hidden_size)

prompt = "my name is george and i am living here in greece, i am 20 years old and my secret is that i am in love"
layer_idx = 8  # layer at which we want to compute the gradient


tokenized = tokenizer(prompt)

# h = get_hidden_representation(prompt, layer_idx, model, tokenizer)
h = get_whole(prompt, model, tokenizer, layer_idx)
if h.dim() == 1:
    h = h.unsqueeze(0)

gamma = 1e-0  # step size for gradient descent
n_iterations = 5000  # number of iterations for gradient descent

discovered_ids = []
for j in range(h.size(0)):
    # an idea here is to initialize with feeding an LLM with the prompt so far
    # and getting the next token, more expensive, likely to work much better
    y_i_id, x_i_plus_1 = get_random_token_id_and_embedding(embedding_matrix)

    for iteration in (bar := tqdm(range(n_iterations), desc=f'Token [{j:2d}/{h.size(0):2d}]')):
        input_ids = torch.tensor(discovered_ids + [y_i_id], dtype=torch.long)

        h_pred = get_whole('', model, tokenizer, layer_idx, input_ids.unsqueeze(0), grad=False)[-1]

        # if torch.sum((h_pred - h[:h_pred.size(0),:]) ** 2) <= 1e-10:
        #     print('Early stopping')
        #     break

        grad_oracle, loss = compute_last_token_embedding_grad(
            y=input_ids, # turn into x_i_plus_1
            llm=model,
            layer_idx=layer_idx,
            h_target=h[j]
        ) # TODO: Dont use tokens, rather use previous embeddings + x_i_plus_1
        # TODO: non-zero loss even when having the correct y_i_id
        
        string_so_far = tokenizer.decode(input_ids.cpu().tolist(), skip_special_tokens=True)

        bar.set_postfix_str(f"Loss: {loss:.2e} - Gradient norm: {grad_oracle.norm().item():.2e} - String: {string_so_far}")
        
        # if string_so_far == prompt:
        #     break

        if loss < 1e-8:
            # print(f"Found token {y_i} with small L2 norm: {torch.norm(h[j] - embedding_matrix[closest_token_id]).item():.4f}")
            break

        # update the guess using gradient descent
        x_i_plus_1 = x_i_plus_1 - gamma * grad_oracle  # (hidden_size,)
        
        # find the closest token in the embedding space to e_i_plus_1
        distances = torch.norm(embedding_matrix - x_i_plus_1, dim=1)  # (vocab_size,)
        y_i_id = int(torch.argmin(distances))


    
    discovered_ids.append(y_i_id)

final_string = tokenizer.decode(input_ids.cpu().tolist(), skip_special_tokens=True)
print(f"Final discovered tokens: {final_string}")
